In [1]:
import pandas as pd

df = pd.read_csv('./original_urls.csv')
df.head(1)

,url
0,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ba...


In [2]:
urls = list(df['url'])
urls[0]

'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Robbsia_andropogonis/latest_assembly_versions/GCF_000566705.1_Ba3549/GCF_000566705.1_Ba3549_genomic.fna.gz'

In [ ]:
from datasets import load_dataset

test_iter = load_dataset(
    "multi_species_genomes.py",
    split="test",
    streaming=True,     
)

train_iter = load_dataset(
    "multi_species_genomes.py",
    split="train",
    streaming=True,   
)

dev_iter = load_dataset(
    "multi_species_genomes.py",
    split="validation",
    streaming=True,     
)


# first 5 examples
import itertools
for ex in itertools.islice(test_iter, 5):
    print(ex)


{'sequence': 'CGGCTTGGACCTCCACCCAAGACTGTCTGGGAGGTGACGTATTGCACGGGACCTACCTTCCTGGGTCTTGGAGTCAGCCACCTGGCAACGGGTGGTTGGGGCACGGTTTCCGGTCCTGGTTTTCTACCCGTCACTCGGGGAGACAAACAGGTGAAGAGAGAAGTGCACCACTAGACTGAGCAGGGCACCCTCAAGGCCTGGCTTGGCCTATGTTTTTCACCTATTCGCTCTCCCCCCACAATAGCAAGTCAGAGAACGAACAGGTGTGATACAGTAGATTAATCAATTTTAGGACACATCATCAGTAGTGTACCTCAGAGTTATCAGGTGTTTTGGCTTTTAAACCCTATATTTGCAGTGCGGTTGGAAAAATTAAATATAATTACATCTACTAGGAACAATTTTCCAATAAACAAATAAATAAAAGGAGATTCACTCCCAGACAGAGGGACTGGAAGGCCCGATGGCATTGCTTAACAAGCGGTGGCCCGTGCATTACCCTTGATCTTCCTATCCAACCACAACTGTCTGATGACCATTGATGGGGGAAGTCATTGTATGTTCCCCTAACCTAACCCTAACCCTAACCCTAAACCCCAACCCTAAACCTAACCTTGGCCCTAATCTTAGCCCTAAACCTAACCCTAAACCCTAATACCTAACCCTAACTTTAGCCCTAAACCTAACCTTGGCCCTAAACCTAACCTTAGCCCTAATCCTAGCCCTAAACCTAACCTTAAACCCTAATACCTAACCTTAACCCTAACCTTAGCCCTAAACCTAATCCTAAACCCTAACCTTTAAATTAATCATAATTCTGAACTTAATCCTGAACCTAATCTTGGATCTACAACCCAACCTTAAACCTAAATTTAGACTTAGACCTAACCCTTTACACCCTCCCACCCAATCTATGGCCTGGGGACCAGATTATCAGGCCCCTGACCTTAACCAGGGGGGTCCTCAAAGAATCCTGGGAATTTTCC

In [ ]:
from urllib.parse import urlparse

def extract_taxa(url: str) -> tuple[str, str, str]:
    """
    Given an NCBI RefSeq URL of the form
      …/refseq/{group}/{Genus_species[_sub…]}/…fna.gz
    returns (group, Genus, species_epithet).
    """
    path = urlparse(url).path.strip("/")
    parts = path.split("/")

    try:
        idx = parts.index("refseq")
        group           = parts[idx + 1]  
        species_folder  = parts[idx + 2] 
    except (ValueError, IndexError):
        raise ValueError(f"URL not in expected RefSeq format: {url!r}")


    bits  = species_folder.split("_")
    genus = bits[0]
    epithet = "_".join(bits[1:]) if len(bits) > 1 else ""

    return group, genus, epithet


In [5]:
dev_iter

IterableDataset({
    features: ['sequence', 'description', 'start_pos', 'end_pos', 'fasta_url'],
    num_shards: 50
})

In [6]:
train_iter

IterableDataset({
    features: ['sequence', 'description', 'start_pos', 'end_pos', 'fasta_url'],
    num_shards: 750
})

In [7]:
test_iter

IterableDataset({
    features: ['sequence', 'description', 'start_pos', 'end_pos', 'fasta_url'],
    num_shards: 50
})

In [ ]:
# from datasets import load_dataset

# train_iter = load_dataset(
#     "multi_species_genomes.py",
#     split="test",
#     streaming=True,
# )

# seen = set()
# it = iter(train_iter)
# while True:
#     try:
#         ex = next(it)
#     except StopIteration:
#         break
#     url = ex["fasta_url"]
#     if url not in seen:
#         seen.add(url)
#         print(f"Got shard {len(seen):4d}/{train_iter.num_shards}: {url}")
#     if len(seen) >= train_iter.num_shards:
#         break

# print("Done: saw all", len(seen), "shards.")

Got shard    1/50: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/904/425/465/GCF_904425465.1_Colossoma_macropomum/GCF_904425465.1_Colossoma_macropomum_genomic.fna.gz


In [ ]:
first_example = next(iter(train_iter))


print(first_example)

{'sequence': 'CAGCGGCGTCCCCACGCTACTGATGGCACGCACAGACGCACAGGCGGCGGACCTGCTGACGTCCGACATCGATCCCGCCGATCACCCGTTCCTGACCGGCAAGCGGACCGTGGAAGGCTTCTATCGCACGCATGCGGGGCTCGAGCAGGCTATTTCGCGCGGCCTTGCCTATGCCCCCTATGCCGACCTGATCTGGTGTGAAACCGGCAAGCCGGATCTCGCCGAGGCGCGCCGGTTCGCCGAAGCGATCCATGCGCAATTTCCCGGCAAGGCATTGGCCTATAACTGTTCGCCATCGTTCAACTGGAAACGCAATCTCGACGACAAAACGATCGATCGTTTCCAGCATGCGCTGGCCGAAATGGGCTATACGTTCCAGTTCATCACGCTGGCTGGCTTCCACAGCCTGAACTACAGCATGTTCGAGCTGGCGCACGGATATGCCCGCGAGCAAATGCGCGCCTTCGTCACATTGCAGGAAAAGGAGTTCGGGGCGGCGGAACGCGGGTTCACCGCGGTCAAGCACCAGCACGAAGTGGGCACCGGCTATTTCGATGCCGTCACGCAGACCATCGAACGCCAGTCGTCTACGACCGCGCTGGCAGGATCGACGGAAGCCGAGCAGTTCAACTAGGCGGCAAGCCTTATCCGGCGCGTTGCGGGCGCTATGGCTGGCGGACGCCTACAGGCACCCGTCGGCCAGCGCGGCGGGTGCCGAAAATGGGATCGAAAAACGGGTTTCGGGCGCAAAGCGATGTCCGGCGCGACGGGATCCCTCACCGGTATATCAAGACGGGAATCCGCGTGTGGGACAGCACACGTTGCGTTTCGCTGCCCACCAGCATGCGCGCCAGGCCATGGCGTCCGTGAGACGCCATCAGGATCACATCGCAACCCTCGCCTTCCGCCGCCTCGATAATGCCGAGATAGGGGGACGCATGCGTGCTTATGCGGGTATTGCACCGCACGCCACGCGAAGCGGCCAG

In [12]:
urls[0]

'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Robbsia_andropogonis/latest_assembly_versions/GCF_000566705.1_Ba3549/GCF_000566705.1_Ba3549_genomic.fna.gz'

In [ ]:
import itertools

cleaned = {
    'sequence': [], 'description': [], 'start_pos': [], 'end_pos': [], 'fasta_url': [], 'original_url': [], 'group': [], 'genus': [], 'species_epithet': []

}




combined = itertools.chain(train_iter, dev_iter, test_iter)


for idx, example in enumerate(combined):
    if idx > 850:
        break
   
    cleaned['sequence'].append(example['sequence'])
    cleaned['description'].append(example['description'])
    cleaned['start_pos'].append(example['start_pos'])
    cleaned['end_pos'].append(example['end_pos'])
    cleaned['fasta_url'].append(example['fasta_url'])
    cleaned['original_url'].append(urls[idx])
    group, genus, species_epithet = extract_taxa(urls[idx])
    cleaned['group'].append(group)
    cleaned['genus'].append(genus)
    cleaned['species_epithet'].append(species_epithet)


IndexError: list index out of range

In [ ]:
for k, v in cleaned.items():

    if len(v)>850:
        v.pop()
    print(len(v))

850
850
850
850
850
850
850
850
850


In [12]:
cleaned_df = pd.DataFrame(cleaned)
cleaned_df.head(1)

,sequence,description,start_pos,end_pos,fasta_url,original_url,group,genus,species_epithet
0,CAGCGGCGTCCCCACGCTACTGATGGCACGCACAGACGCACAGGCG...,NZ_AYSW01000002.1 Robbsia andropogonis Ba3549 ...,0,6200,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ba...,bacteria,Robbsia,andropogonis


In [16]:
dataset_tyoe = []

for i in range(0, 850):
    if i < 750:
        dataset_tyoe.append('train')
    elif i>=750 and i < 800:
        dataset_tyoe.append('dev')
    else:
        dataset_tyoe.append('test')

In [19]:
dataset_tyoe[749]

'train'

In [21]:
cleaned_df['set'] = dataset_tyoe
cleaned_df.to_csv('genomic_species.csv')

In [ ]:
from datasets import load_dataset

# load each split; this will:
#  - call your _split_generators (which in turn calls dl_manager.download_and_extract on urls.txt)
#  - parse FASTA files, clean & chunk them via clean_sequence, filter_fn, etc.
# train_ds = load_dataset("multi_species_genomes.py", split="train")
# val_ds   = load_dataset("multi_species_genomes.py", split="validation")
test_ds  = load_dataset("multi_species_genomes.py", split="test")

# inspect
print(train_ds[0])

In [7]:
train_ds

NameError: name 'train_ds' is not defined